# 这里又是介绍不同的interfaces的

具体参考https://www.gradio.app/four-kinds-of-interfaces/

In [4]:
import gradio as gr
from transformers import pipeline

generator = pipeline('text-generation', model = 'gpt2')

def generate_text(text_prompt):
  response = generator(text_prompt, max_length = 50, num_return_sequences=5)
  return response[0]['generated_text']

textbox = gr.Textbox()

demo = gr.Interface(generate_text, textbox, textbox)

demo.launch()

Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


d:\anaconda\lib\site-packages\transformers\generation\utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# 别人的例子

感觉用gradio做的很不错的一个例子

https://github.com/liliyehuilianhong/AI-instruction-generator-for-RenPy

具体的实现效果：

![demo15](image/demo15_otherguy.gif)

> 老实说看了它的项目之后自己之后有个想法就是可以规范化自己的prompt输出，最终的结果就是做到和他差不多，不过更通用，希望就是可以学完chatgpt prompt engineering之后就用gradio做一个这样的项目

In [1]:
import gradio as gr
import os



# tag生成
def pmt_sd(style, input_text):
    return f"""masterpiece, best quality, masterpiece,best quality,official art,extremely detailed CG unity 8k 
    wallpaper,{style}{input_text}"""


title2 = "Tag生成"
description2 = (
    "点击 Submit 按钮，生成Tag"
)

pmt_demo = gr.Interface(
    pmt_sd,
    [
        gr.CheckboxGroup(
            ["comic", "real", "corlorful", "artbook", "light", "night", "bright", "animate", "album", "photo", "sketch",
             "science_fiction", "back", "bust", "profile", "guro"
             ],
            label="风格", info="增加风格Tag"
        ),
        gr.Textbox(
            inputs="text", outputs="text", label="描述文本输入", info="提供文本描述"
        )
    ],
    "text",
    title=title2,
    description=description2,
    examples=[
        [["comic", "light"], "school,overlooking the teaching building, the white building is clean, modern and has a "
                             "sense of science and technology"],
        [["corlorful", "animate"], "quiet forest, light through the leaves, halo"]
    ]
)


# Prompt生成
def pmt_chat(style_1, choose_txt, text_num):
    return f"""帮我写一个{style_1}风格的{choose_txt}视觉小说剧本，字数为{text_num}"""


title3 = "Prompt生成"
description3 = (
    "点击 Submit 按钮，生成Prompt"
)

pmp_demo = gr.Interface(
    pmt_chat,
    [
        gr.Dropdown(
            ["魔幻", "玄幻", "写实", "架空", "穿越"], lable="风格", info="选择剧本风格"
        ),
        gr.CheckboxGroup(
            ["校园", "恋爱", "喜剧", "魔法", "都市", "热血", "冒险", "机甲", "日常",
             "美食", "悬疑", "竞技", "体育", "科幻", "侦探", "青春", "古风", "家庭",
             "推理", "恐怖", "泡泡系", "异世界", "伦理"
             ],
            label="内容", info="选择剧本内容"
        ),
        gr.Slider(500, 5000, value=500, label="字数", info="选择字数"),
    ],
    "text",
    title=title3,
    description=description3,
)

# 操作文件
def file_ctl(file_0):
    path = f"{file_0}"
    num = 1
    for file in os.listdir(path):
        os.rename(os.path.join(path, file), os.path.join(path, str(num)) + ".png")
        num = num + 1
    return f"已经完成文件重命名"

title4 = "图片批量重命名"
description4 = (
    "点击 Submit 按钮，批量重命名文件"
)

file_demo = gr.Interface(
    file_ctl,
    [
        gr.Textbox(
            inputs="text", outputs="text", label="路径", info="输入文件路径"
        )
    ],
    "text",
    title=title4,
    description=description4,
)



# 多页面启动
demo = gr.TabbedInterface(([pmt_demo, pmp_demo, file_demo]),
                          ["tag", "prompt", "file"])
demo.launch()

C:\Users\dongl\AppData\Local\Temp/ipykernel_6332/130849220.py:26: UserWarning: You have unused kwarg parameters in Textbox, please remove them: {'inputs': 'text', 'outputs': 'text'}
  gr.Textbox(
C:\Users\dongl\AppData\Local\Temp/ipykernel_6332/130849220.py:55: UserWarning: You have unused kwarg parameters in Dropdown, please remove them: {'lable': '风格'}
  gr.Dropdown(
C:\Users\dongl\AppData\Local\Temp/ipykernel_6332/130849220.py:89: UserWarning: You have unused kwarg parameters in Textbox, please remove them: {'inputs': 'text', 'outputs': 'text'}
  gr.Textbox(
d:\anaconda\lib\site-packages\gradio\blocks.py:922: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(
d:\anaconda\lib\site-packages\gradio\blocks.py:922: UserWarning: api_name predict already exists, using predict_2
  warnings.warn(


Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


## 多个数据流

In [2]:
import gradio as gr

def increase(num):
    return num + 1

with gr.Blocks() as demo:
    a = gr.Number(label="a")
    b = gr.Number(label="b")
    btoa = gr.Button("a > b")
    atob = gr.Button("b > a")
    atob.click(increase, a, b)
    btoa.click(increase, b, a)

demo.launch()

Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


## 多步骤（语音识别+情感分析）

![demo15](image/demo15_senti.png)

In [1]:
from transformers import pipeline

import gradio as gr

asr = pipeline("automatic-speech-recognition", "facebook/wav2vec2-base-960h")
classifier = pipeline("text-classification")


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [2]:
def speech_to_text(speech):
    text = asr(speech)["text"]
    return text


def text_to_sentiment(text):
    return classifier(text)[0]["label"]


demo = gr.Blocks()

with demo:
    audio_file = gr.Audio(type="filepath")
    text = gr.Textbox()
    label = gr.Label()

    b1 = gr.Button("Recognize Speech")
    b2 = gr.Button("Classify Sentiment")

    b1.click(speech_to_text, inputs=audio_file, outputs=text)
    b2.click(text_to_sentiment, inputs=text, outputs=label)

demo.launch()


Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


## 连续运行事件Chatbot

In [3]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = random.choice(["How are you?", "I love you", "I'm very hungry"])
        chat_history.append((message, bot_message))
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch()


Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


## 持续运行事件

> 下图的正弦函数是一直在动的，像一个波形图一样

![demo15](image/demo15_sin.png)

In [6]:
import math
import gradio as gr
import plotly.express as px
import numpy as np


plot_end = 2 * math.pi


def get_plot(period=1):
    global plot_end
    x = np.arange(plot_end - 2 * math.pi, plot_end, 0.02)
    y = np.sin(2*math.pi*period * x)
    fig = px.line(x=x, y=y)
    plot_end += 2 * math.pi
    if plot_end > 1000:
        plot_end = 2 * math.pi
    return fig


with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            gr.Markdown("Change the value of the slider to automatically update the plot")
            period = gr.Slider(label="Period of plot", value=1, minimum=0, maximum=10, step=1)
            plot = gr.Plot(label="Plot (updates every half second)")

    dep = demo.load(get_plot, None, plot, every=1)
    period.change(get_plot, period, plot, every=1, cancels=[dep])


if __name__ == "__main__":
    demo.queue().launch()


Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


## 收集事件数据

In [5]:
import gradio as gr

with gr.Blocks() as demo:
    turn = gr.Textbox("X", interactive=False, label="Turn")
    board = gr.Dataframe(value=[["", "", ""]] * 3, interactive=False, type="array")

    def place(board, turn, evt: gr.SelectData):
        if evt.value:
            return board, turn
        board[evt.index[0]][evt.index[1]] = turn
        turn = "O" if turn == "X" else "X"
        return board, turn

    board.select(place, [board, turn], [board, turn])

demo.launch()

Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
